## 1. Load libraries

In [1]:
import pandas as pd
import numpy as np
import os
import importlib
import f_data_processing as dp
import map_data as md

dp = importlib.reload(dp)
print(os.getcwd())


c:\Users\stdt\ETH Zurich\PhD Veronika - 07_Master Thesis Natalie\10_hcm_code\code


## 2. Read surveys and do first filtering

Define the data file paths as dictionary.

In [2]:
survey_dict = {"S0": "../data/Survey_Adaptation Natural Hazards_First Wave_raw data.csv", 
               "S1": "../data/Survey_Adaptation Natural Hazards_Second Wave_raw data.csv"}

Loop through the dictionary and apply filtering and cleansing and type conversion. The following steps are done:

1. Read csv-file and clean column names for trailing spaces, dots, and naming as well as change cell values to numeric for the necessary id-columns.
2. Remove entries that are not complete.
3. Filter out speedies and slowies, and inattentives.
4. Transform needed likert/numeric columns (defined in `map_data.py`) into numeric columns.
5. Transform needed non-numeric cells to numeric value.
6. Translate choice experiment replies (choice descriptions) to english.
7. Drop additional responses where more than two have been done using the same IPA-address.
8. Add prefix to every column name so the different survey can be merged.


In [5]:
df_dict ={}

for s, file in survey_dict.items():
    df = pd.read_csv(file, dtype={"id": "string"}, skiprows=[1,2])
    # remove lines, replace empty cells, rename columnames
    df = df.replace("", pd.NA)
    df.columns = df.columns.str.replace(".", "", regex=False)
    df.columns = df.columns.str.replace(r"\s+", "", regex=True)
    df = dp.rename_columns(df, 'municipality', 'benefits')
    df = dp.to_num_col(df, ['id', 'm'])

    # filter only complete/valid response rows
    df = df[(df["DistributionChannel"] != "preview") & (df["Finished"] != False)].copy()
    df = df[~df["Q_TerminateFlag"].isin(["PoorQuality", "NA", "QuotaMet", "Screened"])].copy()
    
    # Exclude speeders, straightliners, inattentives
    speedy_slowy = dp.rm_speeders(df)
    straightliners = dp.rm_straightliners(df)
    inattentives = df[f'attention_check'] != 'Agree'
    df_filtered = df[~((speedy_slowy | straightliners | inattentives))]
    df_filtered = df_filtered.copy()
    
    # transform liker/numeric scales, non-numeric columns
    df_filtered = dp.string_mapping(df_filtered, md.LIKERT_MAP, md.VALID_COLUMNS, numeric=True)
    
    # remove double ipas
    df_filtered = dp.filter_double_ipa(df_filtered)
    
    # prepare for merging
    df_filtered = df_filtered.add_prefix(f'{s}_')
    df_dict[s] = df_filtered
    print(len(df_filtered))

# merge both surveys and check for uniqueness
merged_waves_df = (
    md.KEYS
    .merge(df_dict['S0'], how="inner", left_on="S0_idx", right_on="S0_id")
    .merge(df_dict['S1'], how="inner", left_on="S1_idx", right_on="S1_m")
    .drop(columns=["S0_id", "S1_m"])
)


print(len(merged_waves_df))


# reset unique ids
df_cleaned = merged_waves_df.reset_index(drop=True)
df_cleaned["respondent_id"] = df_cleaned.index + 1

# map demographics and translate choice options
df_cleaned = dp.string_mapping(df_cleaned, md.DEMOGRAPHICS_DICT, column_patterns=[r"_gender$", r"_age$", r"_education$", r"_income$", r"_language$", r"_language_region$", r"_party_choice$"])
df_cleaned = dp.string_mapping(df_cleaned, md.TRANSLATION_DICT, column_patterns=[r"^S._choice._exemptions", r"^S._choice._costs", r"^S._choice._benefits"])

# %%capture out
# print(len(df_cleaned))

min  150  max  21320
118 respondents were faster than (366.8)s or slower than (2754.999999999999)s
942
min  288  max  10870
60 respondents were faster than (475.55)s or slower than (3196.1000000000004)s
509
504


> This leaves us with TODO 504 respondes that filled out both surveys

In [6]:
df_cleaned.head(10)

,S0_idx,S1_idx,S0_StartDate,S0_EndDate,S0_Status,S0_IPAddress,S0_Progress,S0_duration,S0_Finished,S0_RecordedDate,...,S1_choice9_costs2,S1_choice9_exemptions1,S1_choice9_exemptions2,S1_choice9_benefits1,S1_choice9_benefits2,S1_Q_TerminateFlag,S1_Q_R_Del,S1_screened_out,S1_SelectedLanguage,respondent_id
0,307963907634351,318152052069452,2025-05-08 10:24:25,2025-05-08 10:37:59,IP Address,193.246.191.125,100,814,True,2025-05-08 10:38:00,...,All people pay the same amount,Low- and middle-income earners exempted from c...,Low-income earners exempted from costs,Equal protection levels for all municipalities,Culturally valuable municipalities e.g. with h...,Complete,NaN,False,IT,1
1,307988881480844,318078313779222,2025-05-08 15:59:17,2025-05-08 16:13:18,IP Address,188.60.117.93,100,840,True,2025-05-08 16:13:18,...,People pay proportionally to their income,No groups exempted from costs,Low-income earners exempted from costs,Municipalities most affected by natural hazard...,Economically prosperous municipalities,Complete,NaN,False,FR,2
2,307799977910308,318078398527710,2025-05-06 16:55:34,2025-05-06 17:09:39,IP Address,194.209.36.231,100,844,True,2025-05-06 17:09:40,...,Companies pay proportionally to their CO2 emis...,No groups exempted from costs,No groups exempted from costs,Municipalities in which people have lived in f...,Culturally valuable municipalities e.g. with h...,Complete,NaN,False,DE,3
3,307988976305297,318078398527711,2025-05-08 15:57:09,2025-05-08 16:15:22,IP Address,146.185.81.236,100,1092,True,2025-05-08 16:15:22,...,Companies pay proportionally to their CO2 emis...,Low- and middle-income earners exempted from c...,No groups exempted from costs,Municipalities in which people have lived in f...,Municipalities in which people have lived in f...,Complete,NaN,False,DE,4
4,307799977920554,318078398527712,2025-05-06 12:47:48,2025-05-06 13:22:10,IP Address,195.20.82.203,100,2061,True,2025-05-06 13:22:10,...,People pay proportionally to their CO2 emissions,No groups exempted from costs,Low- and middle-income earners exempted from c...,Culturally valuable municipalities e.g. with h...,Municipalities in which people have lived in f...,Complete,NaN,False,DE,5
5,308339132495007,318078398527713,2025-05-12 15:04:29,2025-05-12 15:32:29,IP Address,83.77.166.207,100,1679,True,2025-05-12 15:32:30,...,People pay proportionally to their CO2 emissions,No groups exempted from costs,No groups exempted from costs,Equal protection levels for all municipalities,Equal protection levels for all municipalities,Complete,NaN,False,DE,6
6,307714469757124,318078398527714,2025-05-05 13:30:31,2025-05-05 13:42:00,IP Address,77.56.75.164,100,689,True,2025-05-05 13:42:01,...,People pay proportionally to their CO2 emissions,No groups exempted from costs,No groups exempted from costs,Equal protection levels for all municipalities,Municipalities in which people have lived in f...,Complete,NaN,False,DE,7
7,307799977928750,318078398527715,2025-05-06 16:45:49,2025-05-06 17:15:43,IP Address,87.245.104.92,100,1793,True,2025-05-06 17:15:43,...,People pay proportionally to their income,Low- and middle-income earners exempted from c...,No groups exempted from costs,Municipalities most affected by natural hazard...,Economically prosperous municipalities,Complete,NaN,False,DE,8
8,307964142323729,318078398527716,2025-05-08 09:48:06,2025-05-08 10:01:43,IP Address,176.223.172.138,100,816,True,2025-05-08 10:01:43,...,People & companies being protected by protecti...,Low- and middle-income earners exempted from c...,Low- and middle-income earners exempted from c...,Economically prosperous municipalities,Municipalities in which people have lived in f...,Complete,NaN,False,DE,9
9,308056942208054,318078398527717,2025-05-11 05:57:34,2025-05-11 06:13:32,IP Address,213.55.241.19,100,957,True,2025-05-11 06:13:32,...,People pay proportionally to their income,Low- and middle-income earners exempted from c...,No groups exempted from costs,Economically prosperous municipalities,Munic

In [7]:

# Attribute-Spalten: Sx_choicey_costs/benefits/exemptionsz
attr_pattern = r"^S\d+_choice\d+_(costs|benefits|exemptions)\d+$"
attr_cols = df_cleaned.filter(regex=attr_pattern).columns

# Präferenz-Spalten: Sx_y_conjoint_prefer
pref_pattern = r"^S\d+_\d+_conjoint_prefer$"
pref_cols = df_cleaned.filter(regex=pref_pattern).columns

# "Restliche" Spalten, die du behalten willst (id, Sozio-Daten etc.)
id_cols = [c for c in df_cleaned.columns if c not in attr_cols and c not in pref_cols]

long_attr = (
    df_cleaned.melt(
        id_vars=id_cols,
        value_vars=attr_cols,
        var_name="var",
        value_name="value"
    )
)

# var sieht z.B. aus wie: S1_choice3_costs2
# x = nh_event, y = choice, Option = Profil (1/2)
long_attr[["nh_event", "choice", "attribute", "option"]] = (
    long_attr["var"]
    .str.extract(r"S(\d+)_choice(\d+)_(costs|benefits|exemptions)(\d+)")
)

# in sinnvolle Typen konvertieren
long_attr["nh_event"] = long_attr["nh_event"].astype(int)
long_attr["choice"]   = long_attr["choice"].astype(int)
long_attr["option"]        = long_attr["option"].astype(int)

long_attr_wide = (
    long_attr
    .pivot(
        index=id_cols + ["nh_event", "choice", "option"],
        columns="attribute",
        values="value"
    )
    .reset_index()
    .copy()
)

long_attr_wide.columns.name = None


pref_long = (
    df_cleaned.melt(
        id_vars=id_cols,
        value_vars=pref_cols,
        var_name="pref_var",
        value_name="preference"
    )
)

# pref_var sieht z.B. so aus: S1_3_conjoint_prefer
pref_long[["nh_event", "choice"]] = (
    pref_long["pref_var"].str.extract(r"S(\d+)_(\d+)_conjoint_prefer")
)

pref_long["nh_event"] = pref_long["nh_event"].astype(int)
pref_long["choice"]   = pref_long["choice"].astype(int)
pref_long = pref_long[id_cols + ["nh_event", "choice", "preference"]]

long_all = long_attr_wide.merge(
    pref_long,
    on=id_cols + ["nh_event", "choice"],
    how="left"
)

long_all = dp.string_mapping(
    long_all,
    mapping_dict=md.preference_map,
    column_patterns=["preference"],
    numeric=True
)

long_all['prefered'] = (long_all['preference'] == long_all['option']).astype(int)

long_all


C:\Users\stdt\AppData\Local\Temp\ipykernel_69848\3770926862.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  .reset_index()
C:\Users\stdt\AppData\Local\Temp\ipykernel_69848\3770926862.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  .reset_index()
C:\Users\stdt\AppData\Local\Temp\ipykernel_69848\3770926862.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) ins

,S0_idx,S1_idx,S0_StartDate,S0_EndDate,S0_Status,S0_IPAddress,S0_Progress,S0_duration,S0_Finished,S0_RecordedDate,...,S1_SelectedLanguage,respondent_id,nh_event,choice,option,benefits,costs,exemptions,preference,prefered
0,307435554211927,318078313782326,2025-05-02 10:09:12,2025-05-02 10:24:20,IP Address,91.132.180.42,100,907,True,2025-05-02 10:24:20,...,FR,79,0,1,1,Municipalities most affected by natural hazard...,Companies pay proportionally to their CO2 emis...,No groups exempted from costs,1,1
1,307435554211927,318078313782326,2025-05-02 10:09:12,2025-05-02 10:24:20,IP Address,91.132.180.42,100,907,True,2025-05-02 10:24:20,...,FR,79,0,1,2,Economically prosperous municipalities,People pay proportionally to their CO2 emissions,Low-income earners exempted from costs,1,0
2,307435554211927,318078313782326,2025-05-02 10:09:12,2025-05-02 10:24:20,IP Address,91.132.180.42,100,907,True,2025-05-02 10:24:20,...,FR,79,0,2,1,Equal protection levels for all municipalities,People pay proportionally to their CO2 emissions,No groups exempted from costs,2,0
3,307435554211927,318078313782326,2025-05-02 10:09:12,2025-05-02 10:24:20,IP Address,91.132.180.42,100,907,True,2025-05-02 10:24:20,...,FR,79,0,2,2,Municipalities most affected by natural hazard...,People & companies being protected by protecti...,Low-income earners exempted from costs,2,1
4,307435554211927,318078313782326,2025-05-02 10:09:12,2025-05-02 10:24:20,IP Address,91.132.180.42,100,907,True,2025-05-02 10:24:20,...,FR,79,0,3,1,Municipalities most affected by natural hazard...,People & companies being protected by protecti...,No groups exempted from costs,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,308412123900067,318152003806452,2025-05-13 20:30:36,2025-05-13 20:44:06,IP Address,193.5.234.233,100,810,True,2025-05-13 20:44:07,...,DE,425,1,7,2,Culturally valuable municipalities e.g. with h...,People & companies being protected by protecti...,Low-income earners exempted from costs,1,0
15116,308412123900067,318152003806452,2025-05-13 20:30:36,2025-05-13 20:44:06,IP Address,193.5.234.233,100,810,True,2025-05-13 20:44:07,...,DE,425,1,8,1,Equal protection levels for all municipalities,People pay proportionally to their CO2 emissions,No groups exempted from costs,1,1
15117,308412123900067,318152003806452,2025-05-13 20:30:36,2025-05-13 20:44:06,IP Address,193.5.234.233,100,810,True,2025-05-13 20:44:07,...,DE,425,1,8,2,Culturally valuable municipalities e.g. with h...,All people pay the same amount,No groups exempted from costs,1,0
15118,308412123900067,318152003806452,2025-05-13 20:30:36,2025-05-13 20:44:06,IP Address,193.5.234.233,100,810,True,2025-05-13 20:44:07,...,DE,425,1,9,1,Municipalities in which people have lived in f...,Companies pay proportionally to their CO2 emis...,No groups exempted from costs,2,0
